## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,69.80,87,1,16.11,clear sky
1,1,Mataura,NZ,-46.1927,168.8643,44.47,91,96,3.31,overcast clouds
2,2,Praia Da Vitoria,PT,38.7333,-27.0667,70.75,94,0,2.35,clear sky
3,3,Ulcinj,ME,41.9294,19.2244,76.50,51,0,3.80,clear sky
4,4,Punta Arenas,CL,-53.1500,-70.9167,32.11,86,20,16.11,few clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,69.80,87,1,16.11,clear sky
2,2,Praia Da Vitoria,PT,38.7333,-27.0667,70.75,94,0,2.35,clear sky
3,3,Ulcinj,ME,41.9294,19.2244,76.50,51,0,3.80,clear sky
8,8,Puerto Ayora,EC,-0.7393,-90.3518,67.55,98,47,1.99,scattered clouds
13,13,Rikitea,PF,-23.1203,-134.9692,74.21,79,99,23.64,light rain
14,14,Saint-Philippe,RE,-21.3585,55.7679,65.79,76,19,11.86,few clouds
15,15,Ust-Kuyga,RU,70.0167,135.6000,71.91,40,7,7.18,clear sky
19,19,Bria,CF,6.5423,21.9863,69.60,96,100,1.77,light rain
21,21,Touros,BR,-5.1989,-35.4608,72.55,91,100,9.08,overcast clouds
22,22,Luanda,AO,-8.8368,13.2343,66.78,84,92,6.24,overcast clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,69.80,clear sky,-22.9661,-42.0278,
2,Praia Da Vitoria,PT,70.75,clear sky,38.7333,-27.0667,
3,Ulcinj,ME,76.50,clear sky,41.9294,19.2244,
8,Puerto Ayora,EC,67.55,scattered clouds,-0.7393,-90.3518,
13,Rikitea,PF,74.21,light rain,-23.1203,-134.9692,
14,Saint-Philippe,RE,65.79,few clouds,-21.3585,55.7679,
15,Ust-Kuyga,RU,71.91,clear sky,70.0167,135.6000,
19,Bria,CF,69.60,light rain,6.5423,21.9863,
21,Touros,BR,72.55,overcast clouds,-5.1989,-35.4608,
22,Luanda,AO,66.78,overcast clouds,-8.8368,13.2343,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,69.80,clear sky,-22.9661,-42.0278,Pousada Porto Praia
2,Praia Da Vitoria,PT,70.75,clear sky,38.7333,-27.0667,Hotel Branco I
3,Ulcinj,ME,76.50,clear sky,41.9294,19.2244,Hotel Palata Venezia
8,Puerto Ayora,EC,67.55,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Rikitea,PF,74.21,light rain,-23.1203,-134.9692,People ThankYou


In [24]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))